## Как происходит обучение
Шаги, выполняемые в так называемом цикле обучения, который повторяется необходимое количество раз.
1. Извлекается пакет обучающих экземпляров x и соответствующих целей y_true.
2. Модель обрабатывает пакет x (этот шаг называется прямым проходом) и получает пакет предсказаний y_pred.
3. Вычисляются потери модели на пакете, дающие оценку несовпадения между y_pred и y_true.
4. Веса модели корректируются так, чтобы немного уменьшить потери на этом пакете.

В конечном итоге получается модель, имеющая очень низкие потери на обучающем наборе данных: несовпадение предсказаний y_pred с ожидаемыми целями y_true малое. Модель «научилась» отображать входные данные в правильные конечные значения. Со стороны все это может походить на волшебство, однако, если разобрать процесс на мелкие шаги, он выглядит очень просто.

**Градиентный спуск — метод оптимизации, широко применимый в современных нейронных сетях. Суть его заключается в следующем: все функции, используемые в наших моделях (например, dot или +), плавно и непрерывно преобразуют свои входные данные. Например, небольшое изменение y в операции z = x + y приведет к небольшому изменению z — и, зная направление изменения y, можно определить направление изменения z. Говоря математическим языком, данные функции дифференцируемы. Если объединить их в цепочку, получившаяся общая функция все равно будет дифференцируемой. Это утверждение, в частности, верно для функции, сопоставляющей веса модели с потерями в пакете данных. Небольшое изменение весов приводит к небольшому и предсказуемому изменению значения потерь, что позволяет использовать математический оператор, называемый градиентом, для описания изменения потерь при изменении весов модели в разных направлениях. Вычисленный градиент можно использовать для модификации весов (всех сразу в одном цикле, а не по одному) в направлении, уменьшающем потери.**

* Поскольку функция непрерывна, небольшое изменение x может дать в результате только небольшое изменение y — это вытекает из понятия непрерывности.

### Стохастический градиентный спуск

**Стохастический градиентный спуск на небольших пакетах (mini-batch stochastic gradient descent, mini-batch SGD). Термин «стохастический» отражает тот факт, что каждый пакет данных выбирается случайно (в науке слово «стохастический» считается синонимом слова «случайный»).**

1. Извлекается пакет обучающих экземпляров x и соответствующих целей y_true.
2. Модель обрабатывает пакет x и получает пакет предсказаний y_pred.
3. Вычисляются потери модели на пакете, дающие оценку несовпадения между y_pred и y_true.
4. Вычисляется градиент потерь для весов модели (обратный проход).

Веса модели корректируются на небольшую величину в направлении, противоположном градиенту (например, W -= скорость_обучения * градиент), и тем самым снижаются потери. Скорость обучения — скалярный множитель, модулирующий «скорость» процесса градиентного спуска.

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

Теперь вам известно, что входные изображения хранятся в тензорах NumPy типа float32, имеющих форму (60000, 784) (обучающие данные) и (10000, 784) (контрольные данные) соответственно.

In [10]:
model = tf.keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [11]:
# sparse_categorical_crossentropy — это функция потерь, которая используется в качестве сигнала обратной связи для обучения весовых тензоров и которую этап обучения стремится свести к минимуму. Вы также знаете, что снижение потерь достигается за счет применения алгоритма стохастического градиентного спуска на небольших пакетах. Точные правила, управляющие конкретным применением градиентного спуска, определяются оптимизатором rmsprop, который передается в первом аргументе.

model.compile(optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

In [12]:
# модель начинает перебирать обучающие данные мини-пакетами по 128 образцов и выполняет пять итераций (каждая итерация по всем обучающим данным называется эпохой).
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2607 - accuracy: 0.9244
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1060 - accuracy: 0.9689
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0706 - accuracy: 0.9785
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0503 - accuracy: 0.9850
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0375 - accuracy: 0.9888


#### В течение пяти эпох сеть выполнит 2345 изменений градиента (по 469 на эпоху), после чего потери модели окажутся достаточно низкими, чтобы она могла классифицировать рукописные цифры с высокой точностью.